In [1303]:
import pandas as pd
from scipy.optimize import minimize
import plotly.graph_objects as go
import numpy as np
ln = np.log  

In [1304]:
df = pd.read_excel("extrem.xlsx")

In [1305]:
df.head()

,Datetime,Unnamed: 1,temp,Unnamed: 3,tnz,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,tt,t_tenzo,tnzl,dif
0,2024-03-18 14:06:40,8,34.6248,26.800,2183140,2178584,2178557,0,0,42.61,36.4642,26,52.15,0,-163,35.108667,2183140,0
1,2024-03-18 14:06:52,8,35.1197,24.120,2183170,2178549,2178525,0,0,38.31,36.9437,24,52.47,0,-861,35.574000,2183170,30
2,2024-03-18 14:08:05,8,35.5940,21.708,2183159,2178476,2178503,0,0,37.17,37.4262,-27,52.73,0,-1603,36.068667,2183159,-11
3,2024-03-18 14:09:17,8,36.0510,19.537,2183152,2178408,2178484,0,0,37.02,37.8699,-75,52.96,0,-2266,36.510667,2183152,-7
4,2024-03-18 14:10:30,8,36.4945,17.584,2183170,2178368,2178466,0,0,37.15,38.3060,-98,53.17,0,-2935,36.956667,2183170,18


In [1306]:
desired_columns = ['Datetime','t_tenzo','tnzl']

In [1307]:
df = df.drop(df.columns.difference(desired_columns),axis=1)

In [1308]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 909 entries, 0 to 908
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Datetime  909 non-null    datetime64[ns]
 1   t_tenzo   909 non-null    float64       
 2   tnzl      909 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 21.4 KB


In [1309]:
df.head()

,Datetime,t_tenzo,tnzl
0,2024-03-18 14:06:40,35.108667,2183140
1,2024-03-18 14:06:52,35.574000,2183170
2,2024-03-18 14:08:05,36.068667,2183159
3,2024-03-18 14:09:17,36.510667,2183152
4,2024-03-18 14:10:30,36.956667,2183170


In [1310]:
def create_plot(df):
    df.loc[150:, 'tnzl'] += 80

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=df['Datetime'], y=df['tnzl'], mode='lines', name='tenzo'))
    fig.add_trace(go.Scatter(x=df['Datetime'], y=df['t_tenzo'], mode='lines', name='Temp', yaxis='y2'))

    fig.update_layout(
        title='График Temp и tenzo от времени',
        title_x=0.5,
        xaxis=dict(title='Datetime'),
        yaxis=dict(tickformat='d', title='Tenzo', side='left', visible=True),
        yaxis2=dict(tickformat='d', title='Temp', overlaying='y', side='right', visible=True),
        hovermode="x",
        font=dict(size=14),
        margin=dict(l=100, r=100, t=50, b=50)
    )

    fig.show()


In [1311]:
create_plot(df)

In [1312]:
delta_tnz = np.nanmax(df.tnzl) - np.nanmin(df.tnzl)
print("Разница уровней =", delta_tnz)

Разница уровней = 1710


In [1313]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 909 entries, 0 to 908
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Datetime  909 non-null    datetime64[ns]
 1   t_tenzo   909 non-null    float64       
 2   tnzl      909 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 21.4 KB


In [1314]:
pearson_corr = df['t_tenzo'].corr(df['tnzl'], method='pearson')
spearman_corr = df['t_tenzo'].corr(df['tnzl'], method='spearman')

print("Коэффициент Пирсона:", pearson_corr)
print("Коэффициент Спирмена:", spearman_corr)

Коэффициент Пирсона: 0.781616267689445
Коэффициент Спирмена: 0.8426040093870645


In [1315]:
def objective_function(C, df):

    t_exp = df.t_tenzo * (1 - np.exp(-abs(C[0])))  

    U_ln = df['tnzl'] - C[2] * ln(t_exp)

    dt = df['t_tenzo'].diff().fillna(0) * C[1]

    dU = U_ln.diff().fillna(0) * C[3]

    dU_meam_dt = dU + C[4]*dt
    
    dU_meam_dt = dU_meam_dt.fillna(0)
    
    dU_meam_dtt = U_ln + dU_meam_dt * C[5]
    
    valid_seconds = list(range(1, 910))

    U_ln_dU_meam_dt = dU_meam_dtt - C[6] * ln(valid_seconds)

    dU_ln_dU_meam_dt = U_ln_dU_meam_dt.diff().fillna(0) 

    result_integrall = U_ln_dU_meam_dt - dU_ln_dU_meam_dt*C[7]

    var_level = np.var(result_integrall)
    
    return var_level

initial_guess = [3, 2.43, 2791, 0.000027, 0.000096, 49.39, 34.27, 0.5]

result = minimize(objective_function, initial_guess, args=(df,), method='BFGS')

optimal_K = result.x
optimal_scalar_value = objective_function(optimal_K, df)

print('Оптимальные значения K:', pd.DataFrame(optimal_K))
print('Оптимальные значения дисперсии:', optimal_scalar_value)


Оптимальные значения K:              0
0     5.800495
1     5.317710
2  2054.455074
3     0.137972
4    10.021547
5    29.680073
6   117.614511
7     0.497372
Оптимальные значения дисперсии: 2446.2258930526623


In [1316]:
def function(C, df):

    t_exp = df.t_tenzo * (1 - np.exp(-abs(C[0])))  
    print("t_exp", t_exp[0:4])
    U_ln = df['tnzl'] - C[2] * ln(t_exp)
    print("U_ln", U_ln[0:4])

    dt = df['t_tenzo'].diff().fillna(0) * C[1]
    print("dt", dt[0:4])

    dU = U_ln.diff().fillna(0) * C[3]
    print("dU", dU[0:4])

    dU_meam_dt = dU + C[4]*dt
    print("dU_meam_dt", dU_meam_dt[0:4])

    dU_meam_dt = dU_meam_dt.fillna(0)
    print("dU_meam_dt", dU_meam_dt[0:4])
    
    dU_meam_dtt = U_ln + dU_meam_dt * C[5]
    print("dU_meam_dtt", dU_meam_dtt[0:4])
    
    valid_seconds = list(range(1, 910))

    U_ln_dU_meam_dt = dU_meam_dtt - C[6] * ln(valid_seconds)
    print("U_ln_dU_meam_dt", U_ln_dU_meam_dt[0:4])

    dU_ln_dU_meam_dt = U_ln_dU_meam_dt.diff().fillna(0)
    print("dU_ln_dU_meam_dt", dU_ln_dU_meam_dt[0:4])

    result_integrall = U_ln_dU_meam_dt - dU_ln_dU_meam_dt*C[7]
    print("result_integrall", result_integrall[0:4])

    return result_integrall

level_dHdt = function(result.x,df)

optimal_K_list = [optimal_K]
optimal_K_df = pd.DataFrame(optimal_K_list,columns=['K{}'.format(i) for i in range(0, 8)])
optimal_K_df.to_csv('K0.csv', mode='a', header=False, index=False)


t_exp 0    35.002426
1    35.466351
2    35.959521
3    36.400183
Name: t_tenzo, dtype: float64
U_ln 0    2.175836e+06
1    2.175839e+06
2    2.175799e+06
3    2.175767e+06
dtype: float64
dt 0    0.000000
1    2.474508
2    2.630494
3    2.350428
Name: t_tenzo, dtype: float64
dU 0    0.000000
1    0.406873
2   -5.432098
3   -4.418296
dtype: float64
dU_meam_dt 0     0.000000
1    25.205269
2    20.929522
3    19.136628
dtype: float64
dU_meam_dt 0     0.000000
1    25.205269
2    20.929522
3    19.136628
dtype: float64
dU_meam_dtt 0    2.175836e+06
1    2.176587e+06
2    2.176420e+06
3    2.176335e+06
dtype: float64
U_ln_dU_meam_dt 0    2.175836e+06
1    2.176505e+06
2    2.176291e+06
3    2.176172e+06
dtype: float64
dU_ln_dU_meam_dt 0      0.000000
1    669.519007
2   -213.964064
3   -119.071906
dtype: float64
result_integrall 0    2.175836e+06
1    2.176172e+06
2    2.176398e+06
3    2.176231e+06
dtype: float64


In [1317]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df['Datetime'], y=df.tnzl, mode='lines', name='Tenzo Data'))
fig.add_trace(go.Scatter(x=df['Datetime'], y=level_dHdt, mode='lines', name='Level Data', yaxis='y2'))
    
  
fig.update_layout(
    legend=dict(x=.8),
    title='График сравнения  Tenzo и Level',
    title_x=0.5,
    xaxis=dict(title='Datatime'),
    yaxis=dict(tickformat ='d',title='Tenzo Data', side='left'),
    yaxis2=dict(tickformat ='d',title='Level Data', overlaying='y', side='right'),
    hovermode="x",
    font=dict(size=14),
    margin=dict(l=100, r=100, t=50, b=50) 
    )
fig.show()


In [1318]:
delta_level = np.max(level_dHdt) - np.min(level_dHdt)
print("Разница уровней =", delta_level)
print("Дисперсия = ", np.var(level_dHdt))


Разница уровней = 604.2170491446741
Дисперсия =  2446.2258930526623


In [1319]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df['Datetime'], y=df.tnzl, mode='lines', name='Tenzo Data'))
fig.add_trace(go.Scatter(x=df['Datetime'], y=level_dHdt, mode='lines', name='Level Data'))
    
  
fig.update_layout(
    legend=dict(x=.8),
    title='График сравнения  Tenzo и Level',
    title_x=0.5,
    xaxis=dict(title='Datatime'),
    yaxis=dict(tickformat ='d',title='Tenzo Data', side='left'),
    hovermode="x",
    font=dict(size=14),
    margin=dict(l=100, r=100, t=50, b=50) 
    )
fig.show()
